In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
import hdbscan
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
import numpy as np
from sklearn.metrics.cluster import contingency_matrix
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from ucimlrepo import fetch_ucirepo 
  
adult = fetch_ucirepo(id=2) 
X = adult.data.features 
y = adult.data.targets 

df = X.copy()
df['y'] = y
 
df=df.dropna()

df['target'] = df['y'].map({'<=50K': 0, '<=50K.': 0, '>50K': 1, '>50K.': 1})
df = df.drop('y', axis=1)

df_num = df[['age', 'fnlwgt', 'capital-gain', 'capital-loss', 'hours-per-week', 'target']]

def sample_from_group(group):
    return group.sample(min(len(group), 5000), random_state=42)

sampled_df = df_num.groupby('target', group_keys=False).apply(sample_from_group)
sampled_df.reset_index(drop=True, inplace=True)

scaler = MinMaxScaler()
df_num_scaled = scaler.fit_transform(sampled_df)

df_num = pd.DataFrame(df_num_scaled, columns=df_num.columns)

target_counts = df_num['target'].value_counts()
print(target_counts)

num_observations = df_num.shape[0]
print("Number of Observations:", num_observations)

In [ ]:
measure = df_num.copy()
grouped = measure.groupby('target')

stats_dict = {}

# Iterate over each class
for group_name, group_df in grouped:
    # Calculate the mean and standard deviation for each column in the class
    mean_values = group_df.mean()
    std_values = group_df.std()
    
    # Store mean and std in a dictionary for each feature
    for feature in group_df.columns:
        if feature not in stats_dict:
            stats_dict[feature] = {}
        
        stats_dict[feature]['mean_' + str(group_name)] = mean_values[feature]
        stats_dict[feature]['std_' + str(group_name)] = std_values[feature]

print(stats_dict)

In [ ]:
feature_to_delete = 'target'

if feature_to_delete in stats_dict:
    del stats_dict[feature_to_delete]
    print(f"{feature_to_delete} deleted from stats_dict")
else:
    print(f"{feature_to_delete} does not exist in stats_dict")

In [ ]:
rf_dict = {}

# Iterate over each feature in stats_dict
for feature, stats_values in stats_dict.items():
    # Calculate rf_i value using the f1 formula
    rf_i = ((stats_values['mean_0.0'] - stats_values['mean_1.0'])**2) / (stats_values['std_0.0']**2 + stats_values['std_1.0']**2)
    rf_dict[feature] = rf_i

print(rf_dict)

In [ ]:
# inverse f1 score
f1_overlap = 1/(1+0.18492082592324244)   #max value in rf_dict
print(f1_overlap)